In [4]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [5]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


In [ ]:
text = "Write or paste your long input text here that you want to summarize."


In [ ]:
tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")


In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

def text_summarizer(text, max_length=150, min_length=50):
    # Load pre-trained BART model and tokenizer
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Encode the input text
    inputs = tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=4, early_stopping=True)

    # Decode the summary tokens back to text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

if __name__ == '__main__':
    input_text = """
    Write or paste your long input text here that you want to summarize.
    It can be several paragraphs or a long article. The summary will condense it.
    """

    summary = text_summarizer(input_text)
    print("Summary:")
    print(summary)


In [2]:
# Import necessary libraries
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch.optim as optim
import torch.nn as nn
from datasets import load_dataset
from torch.utils.data import DataLoader

# Load the XSum dataset
dataset = load_dataset("xsum")

# Initialize the Pegasus tokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

# Preprocess the dataset
def preprocess_function(examples):
    inputs = examples["document"]
    targets = examples["summary"]
    inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(targets, truncation=True, padding="max_length", max_length=150)
    return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"], "labels": targets["input_ids"]}

preprocessed_dataset = dataset.map(preprocess_function, batched=True)

# Configure training settings
batch_size = 4
learning_rate = 1e-5
num_epochs = 5

# Load the Pegasus model
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# Define DataLoader
dataloader = DataLoader(preprocessed_dataset["train"], batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    total_loss = 0.0
    
    for batch in dataloader:
        inputs = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        
        optimizer.zero_grad()
        outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")


ImportError: cannot import name 'load_dataset' from 'datasets' (C:\Users\Leinad\AppData\Roaming\Python\Python311\site-packages\datasets\__init__.py)